In [2]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

# module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if module_path not in sys.path:
#     sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
# if module_path not in sys.path:
#     sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced
import  network_models.w2v_emotion_model.custom_collator as cc
import network_models.w2v_emotion_model.custom_model as cm
import network_models.w2v_emotion_model.model_trainer as ct
import gc

model_name_or_path = "facebook/wav2vec2-large-960h-lv60-self"
pooling_mode = "mean"
device = "cuda"
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                             directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                             directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                             directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                             device="cpu")

emo_dataset = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device=device)
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
gc.collect()
model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device)
model.freeze_feature_extractor()


trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)
#trainSet, evalSet = ADU.train_val_dataset(evalSet, 0.1)


data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))

# trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=2, save_model_every=500, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
# gc.collect()
# trainer.train()


2023-02-02 18:24:05.908824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 18:24:06.488487: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-02 18:24:06.488538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-02 18:24:06.488544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:
#model.load_state_dict(torch.load("../content/classifier/W2VClassifier/Nr1/emo_reco_950.pth"))
#model.from_pretrained()

In [3]:
importlib.reload(ct)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=4, save_model_every=100, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
gc.collect()
trainer.train()

Epoch 1
-------------------------------


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/network_models/w2v_emotion_model/custom_model.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = softmax(y)


loss: 1.983259  [    0/ 2482]
loss: 1.957431  [  400/ 2482]
loss: 1.959857  [  800/ 2482]
loss: 1.908865  [ 1200/ 2482]
loss: 1.880075  [ 1600/ 2482]
loss: 1.980161  [ 2000/ 2482]
loss: 1.875693  [ 2400/ 2482]
Test Error: 
 Accuracy: 19.9%, Avg loss: 1.936945 

Epoch 2
-------------------------------
loss: 1.925819  [    0/ 2482]
loss: 1.869779  [  400/ 2482]
loss: 1.906779  [  800/ 2482]
loss: 1.890250  [ 1200/ 2482]
loss: 1.876533  [ 1600/ 2482]
loss: 1.859152  [ 2000/ 2482]
loss: 1.910106  [ 2400/ 2482]
Test Error: 
 Accuracy: 22.8%, Avg loss: 1.920455 

Epoch 3
-------------------------------
loss: 1.987644  [    0/ 2482]
loss: 1.732911  [  400/ 2482]
loss: 2.015997  [  800/ 2482]
loss: 1.834145  [ 1200/ 2482]
loss: 1.795798  [ 1600/ 2482]
loss: 1.825293  [ 2000/ 2482]
loss: 2.025241  [ 2400/ 2482]
Test Error: 
 Accuracy: 24.6%, Avg loss: 1.909152 

Epoch 4
-------------------------------
loss: 1.752496  [    0/ 2482]
loss: 1.967472  [  400/ 2482]
loss: 1.887406  [  800/ 2482]
loss

KeyboardInterrupt: 